# Notebook to calculate the rotation of the MIRI images with respect to NIRCam
Let's start with the imports

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.ndimage import rotate
from reproject import reproject_interp

Now we define a function that given an input .fits file calculates the angle needed to correct the rotation

In [ ]:
def calculate_angle(fits_file):
    """A function that reads in the header of a .fits file and extracts the information
        about the rotation of the image with respect to the N and E directions.

    Args:
        fits_file (string): The .fits file to be rotated in the next steps
    """
    with fits.open(fits_file) as hdul:
        header = hdul[1].header

        # Extract rotation angle from PC matrix
        if 'PC1_1' in header and 'PC2_2' in header:
            pc11 = header['PC1_1'] 
            pc12 = header['PC1_2'] 
            pc21 = header['PC2_1'] 
            pc22 = header['PC2_2'] 
            
            angle = 180 - np.arccos(header['PC1_1']) * 180 / np.pi
            print(f"The image {fits_file} is rotated by {angle:.2f} degrees with respect to North")        
        else:
            print("No PC matrix found, assuming no rotation")
            angle = 0
        
    return angle

Define a function that takes a .fits file, extends the pixel domain and rotates it according to some angle specified by the user.

In [ ]:
def expand_fits(fits_file, output_folder, angle):
    """
    Expands the FITS image to prevent cropping when rotated,
    updates the WCS, and saves the new expanded FITS file.

    Args:
        fits_file (str): Path to the input FITS file.
        output_folder (str): Directory to save the expanded FITS file.
        angle (float): The rotation angle in degrees (used to determine expansion size).

    Returns:
        str: Path to the expanded FITS file.
    """
    with fits.open(fits_file) as hdul:
        image_data = hdul[1].data
        header = hdul[1].header
        wcs = WCS(header)

        # Original dimensions
        ny, nx = image_data.shape

        # Compute the new bounding box size to avoid cropping
        theta = np.radians(angle)
        cos_t, sin_t = np.abs(np.cos(theta)), np.abs(np.sin(theta))
        new_nx = int(nx * cos_t + ny * sin_t)
        new_ny = int(nx * sin_t + ny * cos_t)

        # Calculate padding needed
        padx = (new_nx - nx) // 2
        pady = (new_ny - ny) // 2

        # Expand the image with NaN values
        expanded_image = np.full((ny + 2 * pady, nx + 2 * padx), np.nan, dtype=np.float32)
        expanded_image[pady:pady+ny, padx:padx+nx] = image_data

        # Adjust WCS: Update reference pixel position
        header['CRPIX1'] += padx
        header['CRPIX2'] += pady
        
        # Generate output filename
        output_file = os.path.join(output_folder, os.path.splitext(os.path.basename(fits_file))[0] + '_exp.fits')        

        # Save the expanded image
        hdu = fits.PrimaryHDU(expanded_image, header=header)
        hdu.writeto(output_file, overwrite=True)
        print(f"Saved expanded FITS file to {output_file}")
    
    return output_file

Define a function to rotate and project the .fits file onto the new frame while preserving the WCS coordinates and save it to a new file.

In [ ]:
def rotate_exp(fits_file, output_dir, angle):
    """A function that takes in an expanded .fits file and performs the rotation about
    a certain angle specified when calling the function.

    Args:
        expanded_file (string): The expanded .fits file to be rotated
        output_dir (string): The output directory to save the rotated .fits file to
        angle (float): The angle of rotation
    """
    with fits.open(fits_file) as hdul:
        hdul.info()
        image_data = hdul[0].data
        header = hdul[0].header
        wcs = WCS(header)

        # Get original reference pixel (before rotation)"
        crpix_original = np.array([header['CRPIX1'], header['CRPIX2']])
        print("CRPIX1:", header['CRPIX1'], "CRPIX2:", header['CRPIX2'])
        print("Image shape:", image_data.shape)
        print(crpix_original)
 
        # Get world coordinates of the original reference pixel (before rotation)
        crval_original = wcs.pixel_to_world(crpix_original[0], crpix_original[1])
        
        # Update the WCS: Rotate PC matrix by -angle (to match sky rotation)
        theta = np.radians(-angle)  # Convert to radians (negative for counterclockwise)
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        if 'PC1_1' in header and 'PC2_2' in header:
            pc_matrix = np.array([[header['PC1_1'], header['PC1_2']],
                                  [header['PC2_1'], header['PC2_2']]])
            rotation_matrix = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
            new_pc_matrix = np.dot(rotation_matrix, pc_matrix)

        # Update the PC matrix in the header
        header['PC1_1'], header['PC1_2'] = new_pc_matrix[0]
        header['PC2_1'], header['PC2_2'] = new_pc_matrix[1]

        # Recalculate reference pixel position
        new_wcs = WCS(header)
        new_crpix = new_wcs.world_to_pixel(crval_original)

        # Correct for displacement by shifting CRPIX back to original sky position
        header['CRPIX1'] = np.round(new_crpix[0])
        header['CRPIX2'] = np.round(new_crpix[1])

        # Perform the actual action of rotation - this is where the magic happens
        rotated_image = rotate(image_data, angle, reshape=False, order=1, mode='nearest')
        
        # Save the rotated FITS file
        os.makedirs(output_dir, exist_ok=True)
        output_file = os.path.join(output_dir, os.path.basename(fits_file).replace('.fits', '_rot.fits'))
        hdu = fits.PrimaryHDU(rotated_image, header=header)
        hdu.writeto(output_file, overwrite=True)

        print(f"Rotated image saved to {output_file}")

Alternative rotation function using reproject_interp to rotate the image (doesn't work!!!)

In [ ]:
def rotate_interp(fits_file, output_dir, angle):
    """Rotates a FITS image while preserving WCS information using reproject_interp.

    Args:
        fits_file (str): The expanded FITS file to be rotated.
        output_dir (str): The directory to save the rotated FITS file.
        angle (float): The angle of rotation in degrees (counterclockwise).
    """
    with fits.open(fits_file) as hdul:
        hdul.info()
        image_data = hdul[0].data
        header = hdul[0].header
        wcs = WCS(header)

        # Compute the new WCS by applying the rotation
        theta = np.radians(-angle)  # Negative for counterclockwise rotation
        cos_t, sin_t = np.cos(theta), np.sin(theta)

        # Extract and rotate the PC matrix
        if 'PC1_1' in header and 'PC2_2' in header:
            pc_matrix = np.array([
                [header['PC1_1'], header['PC1_2']],
                [header['PC2_1'], header['PC2_2']]
            ])
            rotation_matrix = np.array([[cos_t, -sin_t], [sin_t, cos_t]])
            new_pc_matrix = np.dot(rotation_matrix, pc_matrix)

            # Update the header with the new PC matrix
            header['PC1_1'], header['PC1_2'] = new_pc_matrix[0]
            header['PC2_1'], header['PC2_2'] = new_pc_matrix[1]
        else:
            print("No PC matrix found, assuming no rotation in WCS.")
        
        # Generate new WCS
        new_wcs = WCS(header)

        # Perform reprojected interpolation to rotate the image
        rotated_image, _ = reproject_interp((image_data, wcs), new_wcs, shape_out=image_data.shape)

        # Update CRPIX to align reference pixel correctly
        crval_original = wcs.pixel_to_world(header['CRPIX1'], header['CRPIX2'])
        new_crpix = new_wcs.world_to_pixel(crval_original)
        header['CRPIX1'], header['CRPIX2'] = np.round(new_crpix)

        # Save the rotated FITS file
        output_file = os.path.join(output_dir, os.path.basename(fits_file).replace('.fits', '_rot_int.fits'))
        hdu = fits.PrimaryHDU(rotated_image, header=header)
        hdu.writeto(output_file, overwrite=True)

        print(f"Rotated image saved to {output_file}")

Now after all of this we are finally ready to import out actual data to be rotated!

In [ ]:
# Load PRIMER .fits files
primer = '/home/bpc/University/master/Red_Cardinal/MIRI/PRIMER/'
primer_data = glob.glob(os.path.join(primer, "*.fits"))
print(f"Found {len(primer_data)} FITS files for the PRIMER survey.")

# Rotation needs to be modified 
patch_03_f770w = f"./../MIRI/PRIMER/jw01837-o003_t003_miri_f770w_i2d.fits"
patch_03_f1800w = f"./../MIRI/PRIMER/jw01837-o003_t003_miri_f1800w_i2d.fits"

# Rotation should be fine as it is
patch_04_f770w = f"./../MIRI/PRIMER/jw01837-o004_t004_miri_f770w_i2d.fits"
patch_04_f1800w = f"./../MIRI/PRIMER/jw01837-o004_t004_miri_f1800w_i2d.fits"

# Load COSMOS-Web .fits files - here we will have to loop
cweb = "/home/bpc/University/master/Red_Cardinal/MIRI/COSMOS-Web/"
cweb_data = glob.glob(os.path.join(cweb, "*.fits"))
print(f"Found {len(cweb_data)} FITS files for the COSMOS-Web survey.")

exp_dir = "./../expanded/"

#angle = calculate_angle(patch_04_f770w)
#patch_04_f770w_exp = expand_fits(patch_04_f770w, exp_dir, angle)
#rotate_exp(patch_04_f770w_exp, exp_rot_dir, angle)





# Create and specify output directory for PRIMER
exp_rot_dir = "./../exp_rot/PRIMER/"

for fits_file in primer_data:
    angle = calculate_angle(fits_file)
    # Decreasing the angle of rotation means it is getting rotated less in the clockwise direction!!
    if "003" in fits_file: angle -= 225 # add an additional 180° to the rotation
    exp_fits = expand_fits(fits_file, exp_dir, angle)
    exp_rot_fits = rotate_exp(exp_fits, exp_rot_dir, angle)





# Create and specify output directory for Cosmos-Web
exp_rot_dir = "./../exp_rot/COSMOS-Web/"

for fits_file in cweb_data:
    angle = calculate_angle(fits_file)
    exp_fits = expand_fits(fits_file, exp_dir, angle)
    exp_rot_fits = rotate_exp(exp_fits, exp_rot_dir, angle)
